In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import contextily as ctx
import pyproj
from shapely.geometry import Point, LineString
from zipfile import ZipFile, Path
import datetime

In [2]:
with ZipFile("C:\\Users\\zare\\GEO\\geopanda\\gtfs.zip") as myzip:
    stops_df = pd.read_csv(myzip.open("stops.txt"), dtype={ 
    'stop_id': 'str', 
    'stop_code': 'str',
    'stop_name': 'str',
    'stop_desc' : 'str',                                              
    'stop_lat': 'float',
    'stop_lon': 'float',
    'location_type': 'Int64',
    'parent_station': 'str',
    'wheelchair_boarding': 'str', 
    'platform_code': 'str',
    'zone_id': 'str',
    'level_id' : 'str'
    })
    
    
    stop_times_df = pd.read_csv(myzip.open("stop_times.txt"), dtype={
        'trip_id': 'str',
        'arrival_time': 'str',
        'stop_id': 'str', 
        'departure_time': 'str', 
        'stop_id': 'str',
        'stop_sequence': 'Int64',
        'stop_headsign': 'str',
        'pickup_type': 'Int64',
        'drop_off_type': 'Int64',
    })
    
    
    
    routes_df = pd.read_csv(myzip.open("routes.txt"), dtype={
        'route_id': 'str',  
        'agency_id': 'str',  
        'route_short_name': 'str',  
        'route_long_name': 'str', 
        'route_desc': 'str', 
        'route_type': 'Int64',
        'route_color': 'str',  
        'route_text_color': 'str', 
        'rout_desc': 'str'
    })
    
    trips_df = pd.read_csv(myzip.open("trips.txt"), dtype={
        'route_id': 'str', 
        'service_id': 'str',  
        'trip_id': 'str',
        'shape_id': 'str', 
        'trip_headsign': 'str',
        'trip_short_name': 'str',
        'direction_id': 'Int64',  
        'block_id': 'str',
        'shape_id': 'str',
        'wheelchair_accessible': 'str',  
        'bikes_allowed': 'str'
    })
    
    shapes_df = pd.read_csv(myzip.open("shapes.txt"), dtype={
        'shape_id': 'str', 
        'shape_pt_lat': 'float', 
        'shape_pt_lon': 'float',  
        'shape_pt_sequence': 'Int64'
    })
    
    calendar_df = pd.read_csv(myzip.open("calendar.txt"), dtype={
        'service_id': 'str',  
        'monday': 'bool',  
        'tuesday': 'bool',  
        'wednesday': 'bool',  
        'thursday': 'bool',  
        'friday': 'bool', 
        'saturday': 'bool',  
        'sunday': 'bool',  
        'start_date': 'str', 
        'end_date': 'str',
    })
    
    calendar_dates_df = pd.read_csv(myzip.open("calendar_dates.txt"), dtype={
        'service_id': 'str',  
        'date': 'str',
        'exception_type': 'Int64',
    })
    
    agency_df = pd.read_csv(myzip.open("agency.txt"), dtype={
        'agency_id': 'str', 
        'agency_name': 'str', 
        'agency_url': 'str',  
        'agency_timezone': 'str',
        'agency_lang': 'str', 
        'agency_phone': 'str',
    })

In [3]:
show_date_str = "2023-11-27"

date = datetime.datetime.strptime(show_date_str, "%Y-%m-%d")
date_string = date.strftime("%Y%m%d")
day_of_week_name = date.strftime('%A').lower()

services_for_day_1 = calendar_df[(calendar_df[day_of_week_name]) & (date_string >= calendar_df.start_date) & (date_string <= calendar_df.end_date)].service_id.to_numpy()

services_added_for_day = calendar_dates_df[(calendar_dates_df.date == date_string) & (calendar_dates_df.exception_type == 1)].service_id.to_numpy()
services_removed_for_day = calendar_dates_df[(calendar_dates_df.date == date_string) & (calendar_dates_df.exception_type == 2)].service_id.to_numpy()
services_for_day_2 = np.concatenate([services_for_day_1, services_added_for_day])
services_for_day = np.setdiff1d(services_for_day_2, services_removed_for_day)

trips_for_day = trips_df[trips_df.service_id.isin(services_for_day)]
berlin_bus_route_ids = routes_df[(routes_df['route_type'] == 700) | (routes_df['route_type'] == 3) ].route_id.unique()
day_trip_buses = trips_for_day[trips_for_day.route_id.isin(berlin_bus_route_ids)]

In [4]:
time_windows = [0,24]
cutoffs = time_windows

stop_times_df['arrival_time'] = pd.to_timedelta(stop_times_df['arrival_time'])
stop_times_df['departure_time'] = pd.to_timedelta(stop_times_df['departure_time'])

stop_times_df['arrival_time_in_seconds'] = stop_times_df['arrival_time'].dt.total_seconds()
stop_times_df['departure_time_in_seconds'] = stop_times_df['departure_time'].dt.total_seconds()

In [5]:
def fix_departure_time(times_to_fix):
    
    next_day = times_to_fix >= 24*3600
    times_to_fix[next_day] = times_to_fix[next_day] - 24 * 3600
    
    return times_to_fix

In [6]:
if max(cutoffs) <= 24:
    stop_times_df['departure_time'] = fix_departure_time(stop_times_df.departure_time_in_seconds.values)
    stop_times_df['arrival_time'] = fix_departure_time(stop_times_df.arrival_time_in_seconds.values)

In [7]:
def label_creation(cutoffs):
    
    labels = []
    if max(cutoffs) <= 24:
        for w in cutoffs:
            if float(w).is_integer():
                label = str(w) + ':00'
            else:
                n = math.modf(w)
                label = str(int(n[1])) + ':' + str(int(n[0]*60))
            labels.append(label)
    else:
        labels = []
        for w in cutoffs:
            if float(w).is_integer():
                if w > 24:
                    w1 = w-24
                    label = str(w1) + ':00'
                else:
                    label = str(w) + ':00'
                labels.append(label)
            else:
                if w > 24:
                    w1 = w-24
                    n = math.modf(w1)
                    label = str(int(n[1])) + ':' + str(int(n[0]*60))
                else:
                    n = math.modf(w)
                    label = str(int(n[1])) + ':' + str(int(n[0]*60))
                labels.append(label)

    labels = [labels[i] + '-' + labels[i+1] for i in range(0, len(labels)-1)]

    return labels
        
    

In [8]:
labels = label_creation(cutoffs)

departure_time = stop_times_df.departure_time / 3600
stop_times_df['window'] = pd.cut(departure_time, bins=cutoffs, right=False, labels=labels)

stop_times_df = stop_times_df.loc[~stop_times_df.window.isnull()]
stop_times_df['window'] = stop_times_df.window.astype(str)

In [9]:
day_trip_buses1 = pd.merge(day_trip_buses, stop_times_df[['trip_id','stop_id','arrival_time','departure_time','window']], on='trip_id')

In [10]:
day_trip_buses2 = pd.merge(day_trip_buses1, routes_df[['route_id','route_short_name']], on='route_id')

In [11]:
trip_agg1 = day_trip_buses2.groupby(['stop_id','window','route_short_name'])['trip_id'].nunique().reset_index()

In [12]:
def join_bus_numbers(group):
    return ', '.join(sorted(set(group)))

In [13]:
trip_agg1['BusNum'] = trip_agg1.groupby('stop_id')['route_short_name'].transform(join_bus_numbers)

In [17]:
bus_trips = trip_agg1.groupby(['stop_id','window','BusNum'])['trip_id'].sum().reset_index()

In [19]:
bus_trips

,stop_id,window,BusNum,trip_id
0,000005100145,0:00-24:00,RB66,21
1,000005170001,0:00-24:00,RB66,21
2,000005170002,0:00-24:00,RB66,38
3,000008071192,0:00-24:00,RE3,2
4,000008071193,0:00-24:00,RE3,2
...,...,...,...,...
23298,de:15091:9479::1,0:00-24:00,X2,17
23299,de:15091:9479::2,0:00-24:00,X2,17
23300,de:15091:9480::1,0:00-24:00,X2,17
23301,de:15091:9480::2,0:00-24:00,X2,17


In [20]:
stops_gdf = gpd.GeoDataFrame(stops_df, geometry = gpd.points_from_xy(stops_df.stop_lon, stops_df.stop_lat)).set_crs(epsg=4326)

In [21]:
bus_trips.rename(columns={'trip_id': 'ntrips'}, inplace=True)

In [22]:
start_time = bus_trips.window.apply(lambda x: cutoffs[labels.index(x)])

end_time = bus_trips.window.apply(lambda x: cutoffs[labels.index(x) + 1])

In [23]:
bus_trips['Headway'] = ((end_time - start_time)*60 / bus_trips.ntrips)\
.astype(int)

bus_trips['frequency'] = (bus_trips.ntrips / (end_time - start_time))\
.astype(float)

In [24]:
bus_stop_frequency = pd.merge(bus_trips, stops_gdf[['stop_id','stop_name','geometry']], on='stop_id')

In [25]:
bus_stop_frequency

,stop_id,window,BusNum,ntrips,Headway,frequency,stop_name,geometry
0,000005100145,0:00-24:00,RB66,21,68,0.875000,"Szczecin, Gumience",POINT (14.49486 53.39775)
1,000005170001,0:00-24:00,RB66,21,68,0.875000,"Szczecin, Warzymice Osiedle Kresy",POINT (14.48189 53.38629)
2,000005170002,0:00-24:00,RB66,38,37,1.583333,"Szczecin, Krzysztofa Kolumba",POINT (14.54847 53.41732)
3,000008071192,0:00-24:00,RE3,2,720,0.083333,"Bülzig, Kindergarten",POINT (12.75299 51.89669)
4,000008071193,0:00-24:00,RE3,2,720,0.083333,"Zörnigall, Brücke",POINT (12.73049 51.88334)
...,...,...,...,...,...,...,...,...
23298,de:15091:9479::1,0:00-24:00,X2,17,84,0.708333,"Kropstädt, Gemeinde",POINT (12.74603 51.96441)
23299,de:15091:9479::2,0:00-24:00,X2,17,84,0.708333,"Kropstädt, Gemeinde",POINT (12.74608 51.96429)
23300,de:15091:9480::1,0:00-24:00,X2,17,84,0.708333,"Kropstädt, Feuerwehr",POINT (12.75267 51.96578)
23301,de:15091:9480::2,0:00-24:00,X2,17,84,0.708333,"Kropstädt, Feuerwehr",POINT (12.75263 51.96575)


In [26]:
bus_stop_frequency_gdf = gpd.GeoDataFrame(pd.DataFrame(bus_stop_frequency), geometry='geometry')

bus_stop_frequency_gdf.to_file("C:\\Users\\zare\\GEO\\geopanda\\stop_frequency.shp", driver='ESRI Shapefile')

In [27]:
bus_stop_frequency.ntrips.max()

1249